# Tensorflow FaceNet to Keras model

In [7]:
import os
import re
import numpy as np
import tensorflow as tf

import sys
sys.path.append('../code/')
from inception_resnet_v1 import *

Using TensorFlow backend.


In [19]:
tf_model_dir = './model/tf/20180402-114759/'
npy_weights_dir = './model/keras/npy_weights/'
weights_dir = './model/keras/weights/'
model_dir = './model/keras/model/'

weights_filename = 'facenet_keras_weights.h5'
model_filename = 'facenet_keras.h5'

In [20]:
os.makedirs(npy_weights_dir, exist_ok=True)
os.makedirs(weights_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [21]:
# regex for renaming the tensors to their corresponding Keras counterpart
re_repeat = re.compile(r'Repeat_[0-9_]*b')
re_block8 = re.compile(r'Block8_[A-Za-z]')

def get_filename(key):
    filename = str(key)
    filename = filename.replace('/', '_')
    filename = filename.replace('InceptionResnetV1_', '')

    # remove "Repeat" scope from filename
    filename = re_repeat.sub('B', filename)

    if re_block8.match(filename):
        # the last block8 has different name with the previous 5 occurrences
        filename = filename.replace('Block8', 'Block8_6')

    # from TF to Keras naming
    filename = filename.replace('_weights', '_kernel')
    filename = filename.replace('_biases', '_bias')

    return filename + '.npy'


def extract_tensors_from_checkpoint_file(filename, output_folder):
    reader = tf.train.NewCheckpointReader(filename)

    for key in reader.get_variable_to_shape_map():
        # not saving the following tensors
        if key == 'global_step':
            continue
        if 'AuxLogit' in key:
            continue

        # convert tensor name into the corresponding Keras layer weight name and save
        path = os.path.join(output_folder, get_filename(key))
        arr = reader.get_tensor(key)
        np.save(path, arr)

In [22]:
extract_tensors_from_checkpoint_file(tf_model_dir+'model-20180402-114759.ckpt-275', npy_weights_dir)

In [33]:
model = InceptionResNetV1(classes=512)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
__________________________________________________________________________________________________
Conv2d_2a_

In [34]:
for layer in model.layers:
    if layer.weights:
        weights = []
        for w in layer.weights:
            weight_name = os.path.basename(w.name).replace(':0', '')
            weight_file = layer.name + '_' + weight_name + '.npy'
            weight_arr = np.load(os.path.join(npy_weights_dir, weight_file))
            weights.append(weight_arr)
            print(weight_file, weight_arr.shape)
        layer.set_weights(weights)

#print('Saving weights...')
#model.save_weights(os.path.join(weights_dir, weights_filename))
print('Saving model...')
model.save(os.path.join(model_dir, model_filename))

Conv2d_1a_3x3_kernel.npy (3, 3, 3, 32)
Conv2d_1a_3x3_BatchNorm_beta.npy (32,)
Conv2d_1a_3x3_BatchNorm_moving_mean.npy (32,)
Conv2d_1a_3x3_BatchNorm_moving_variance.npy (32,)
Conv2d_2a_3x3_kernel.npy (3, 3, 32, 32)
Conv2d_2a_3x3_BatchNorm_beta.npy (32,)
Conv2d_2a_3x3_BatchNorm_moving_mean.npy (32,)
Conv2d_2a_3x3_BatchNorm_moving_variance.npy (32,)
Conv2d_2b_3x3_kernel.npy (3, 3, 32, 64)
Conv2d_2b_3x3_BatchNorm_beta.npy (64,)
Conv2d_2b_3x3_BatchNorm_moving_mean.npy (64,)
Conv2d_2b_3x3_BatchNorm_moving_variance.npy (64,)
Conv2d_3b_1x1_kernel.npy (1, 1, 64, 80)
Conv2d_3b_1x1_BatchNorm_beta.npy (80,)
Conv2d_3b_1x1_BatchNorm_moving_mean.npy (80,)
Conv2d_3b_1x1_BatchNorm_moving_variance.npy (80,)
Conv2d_4a_3x3_kernel.npy (3, 3, 80, 192)
Conv2d_4a_3x3_BatchNorm_beta.npy (192,)
Conv2d_4a_3x3_BatchNorm_moving_mean.npy (192,)
Conv2d_4a_3x3_BatchNorm_moving_variance.npy (192,)
Conv2d_4b_3x3_kernel.npy (3, 3, 192, 256)
Conv2d_4b_3x3_BatchNorm_beta.npy (256,)
Conv2d_4b_3x3_BatchNorm_moving_mean.npy 

Block35_5_Branch_1_Conv2d_0b_3x3_kernel.npy (3, 3, 32, 32)
Block35_5_Branch_2_Conv2d_0c_3x3_kernel.npy (3, 3, 32, 32)
Block35_5_Branch_0_Conv2d_1x1_BatchNorm_beta.npy (32,)
Block35_5_Branch_0_Conv2d_1x1_BatchNorm_moving_mean.npy (32,)
Block35_5_Branch_0_Conv2d_1x1_BatchNorm_moving_variance.npy (32,)
Block35_5_Branch_1_Conv2d_0b_3x3_BatchNorm_beta.npy (32,)
Block35_5_Branch_1_Conv2d_0b_3x3_BatchNorm_moving_mean.npy (32,)
Block35_5_Branch_1_Conv2d_0b_3x3_BatchNorm_moving_variance.npy (32,)
Block35_5_Branch_2_Conv2d_0c_3x3_BatchNorm_beta.npy (32,)
Block35_5_Branch_2_Conv2d_0c_3x3_BatchNorm_moving_mean.npy (32,)
Block35_5_Branch_2_Conv2d_0c_3x3_BatchNorm_moving_variance.npy (32,)
Block35_5_Conv2d_1x1_kernel.npy (1, 1, 96, 256)
Block35_5_Conv2d_1x1_bias.npy (256,)
Mixed_6a_Branch_1_Conv2d_0a_1x1_kernel.npy (1, 1, 256, 192)
Mixed_6a_Branch_1_Conv2d_0a_1x1_BatchNorm_beta.npy (192,)
Mixed_6a_Branch_1_Conv2d_0a_1x1_BatchNorm_moving_mean.npy (192,)
Mixed_6a_Branch_1_Conv2d_0a_1x1_BatchNorm_movin

Block17_6_Conv2d_1x1_kernel.npy (1, 1, 256, 896)
Block17_6_Conv2d_1x1_bias.npy (896,)
Block17_7_Branch_1_Conv2d_0a_1x1_kernel.npy (1, 1, 896, 128)
Block17_7_Branch_1_Conv2d_0a_1x1_BatchNorm_beta.npy (128,)
Block17_7_Branch_1_Conv2d_0a_1x1_BatchNorm_moving_mean.npy (128,)
Block17_7_Branch_1_Conv2d_0a_1x1_BatchNorm_moving_variance.npy (128,)
Block17_7_Branch_1_Conv2d_0b_1x7_kernel.npy (1, 7, 128, 128)
Block17_7_Branch_1_Conv2d_0b_1x7_BatchNorm_beta.npy (128,)
Block17_7_Branch_1_Conv2d_0b_1x7_BatchNorm_moving_mean.npy (128,)
Block17_7_Branch_1_Conv2d_0b_1x7_BatchNorm_moving_variance.npy (128,)
Block17_7_Branch_0_Conv2d_1x1_kernel.npy (1, 1, 896, 128)
Block17_7_Branch_1_Conv2d_0c_7x1_kernel.npy (7, 1, 128, 128)
Block17_7_Branch_0_Conv2d_1x1_BatchNorm_beta.npy (128,)
Block17_7_Branch_0_Conv2d_1x1_BatchNorm_moving_mean.npy (128,)
Block17_7_Branch_0_Conv2d_1x1_BatchNorm_moving_variance.npy (128,)
Block17_7_Branch_1_Conv2d_0c_7x1_BatchNorm_beta.npy (128,)
Block17_7_Branch_1_Conv2d_0c_7x1_Batch

Block8_2_Conv2d_1x1_kernel.npy (1, 1, 384, 1792)
Block8_2_Conv2d_1x1_bias.npy (1792,)
Block8_3_Branch_1_Conv2d_0a_1x1_kernel.npy (1, 1, 1792, 192)
Block8_3_Branch_1_Conv2d_0a_1x1_BatchNorm_beta.npy (192,)
Block8_3_Branch_1_Conv2d_0a_1x1_BatchNorm_moving_mean.npy (192,)
Block8_3_Branch_1_Conv2d_0a_1x1_BatchNorm_moving_variance.npy (192,)
Block8_3_Branch_1_Conv2d_0b_1x3_kernel.npy (1, 3, 192, 192)
Block8_3_Branch_1_Conv2d_0b_1x3_BatchNorm_beta.npy (192,)
Block8_3_Branch_1_Conv2d_0b_1x3_BatchNorm_moving_mean.npy (192,)
Block8_3_Branch_1_Conv2d_0b_1x3_BatchNorm_moving_variance.npy (192,)
Block8_3_Branch_0_Conv2d_1x1_kernel.npy (1, 1, 1792, 192)
Block8_3_Branch_1_Conv2d_0c_3x1_kernel.npy (3, 1, 192, 192)
Block8_3_Branch_0_Conv2d_1x1_BatchNorm_beta.npy (192,)
Block8_3_Branch_0_Conv2d_1x1_BatchNorm_moving_mean.npy (192,)
Block8_3_Branch_0_Conv2d_1x1_BatchNorm_moving_variance.npy (192,)
Block8_3_Branch_1_Conv2d_0c_3x1_BatchNorm_beta.npy (192,)
Block8_3_Branch_1_Conv2d_0c_3x1_BatchNorm_moving_m